## Dogs vs. Cats Redux

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
PATH = "../../fastai/courses/dl1/data/dogscats/"
bs=64 # 22 for finetuning with differential LR
sz=299
arch=resnext50

In [4]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs, test_name='test', num_workers=4)
learn = ConvLearner.pretrained(arch, data, precompute=False, ps=0.5)

In [6]:
#learn.unfreeze()
learn.load('299_all_tuned_50')
#learn.load('299_all_tuned_more_50')

### Simple predict

In [7]:
%%time
log_preds = learn.predict(is_test=True)

CPU times: user 3min 39s, sys: 40.6 s, total: 4min 19s
Wall time: 3min 1s


In [8]:
log_preds.shape

(12500, 2)

In [9]:
log_preds_test = log_preds[:,1]
probs_test = np.exp(log_preds_test)
columns = pd.read_csv(f'{PATH}sample_submission.csv', index_col='id').columns
test = pd.DataFrame(probs_test)
test.columns = columns
test.index = [i.split('.jpg')[0].split('/')[-1] for i in data.test_dl.dataset.fnames]
test.index.name ='id'
test.to_csv(f'{PATH}submission3.csv')

### TTA

In [9]:
%%time
log_preds_test = learn.TTA(is_test=True)[0]

CPU times: user 21min 31s, sys: 2min 46s, total: 24min 18s
Wall time: 15min 31s


In [13]:
log_preds_test.shape

(5, 12500, 2)

In [16]:
probs_mean = np.mean(np.exp(log_preds_test),0)
probs_mean.shape

(12500, 2)

In [17]:
probs_dog = probs_mean[:,1]

columns = pd.read_csv(f'{PATH}sample_submission.csv', index_col='id').columns
test = pd.DataFrame(probs_dog)
test.columns = columns
test.index = [i.split('.jpg')[0].split('/')[-1] for i in data.test_dl.dataset.fnames]
test.index.name ='id'
test.to_csv(f'{PATH}submission2.csv')
pd.read_csv(f'{PATH}submission2.csv').head()

,id,label
0,9885,0.999143
1,2701,0.000198
2,11231,0.983915
3,2964,0.999987
4,10651,0.999976


## Train more

In [8]:
learn.freeze()

In [9]:
learn.fit(1e-2, 1, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                      
    0      0.016319   0.013726   0.995605  



[0.013725672, 0.99560546875]

In [10]:
learn.fit(1e-2, 2, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=2), HTML(value='')))

epoch      trn_loss   val_loss   accuracy                      
    0      0.012833   0.014382   0.995605  
    1      0.010544   0.013946   0.995605                      



[0.013945926, 0.99560546875]

In [11]:
learn.save('299_all_tuned_more_50')

## Results

* arch=resnext50, sz=299, no TTA: `0.04909`
* arch=resnext50, sz=299, with TTA: **`0.04820`**
* arch=resnext50, sz=299, no TTA, more epochs: `0.05515`